# Introduction

- Following code is for retrieving data of iitk students from the iitk server using their roll number. <br>
- In roll_no_list, append all the roll numbers whose name and email you want to retrieve from the server. <br>
- If you are outside from IIT Kanpur, then don't forget to use ssl-vpn proxy.
<br><br>
Change Gmail settings for sending bulk mails- https://myaccount.google.com/lesssecureapps

## Import Libraries

In [4]:
from bs4 import BeautifulSoup
import urllib.request as urllib2
import requests
import re
import pandas as pd

import email
import smtplib
import ssl

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

## Extracting Data from IITK Server

In [3]:
s = requests.Session()
s.get("https://oa.cc.iitk.ac.in/Oa/Jsp/Main_Frameset.jsp")
s.get("https://oa.cc.iitk.ac.in/Oa/Jsp/Main_Intro.jsp?frm='SRCH'")
s.get("https://oa.cc.iitk.ac.in/Oa/Jsp/OAServices/IITK_Srch.jsp?typ=stud")


headers1 = {
    "Referer": "https://oa.cc.iitk.ac.in/Oa/Jsp/OAServices/IITk_SrchStudRoll_new.jsp"
}

payload1 = {
    'typ': ['stud'] * 12,
    'numtxt': '',
    'sbm': ['Y'] * 12
}

# filename should be in format: 'xyz.csv'
# roll_no_list: should be an array of roll nos

def getRecordsByRollNoList (filename, roll_no_list):
    record = []
    for roll in roll_no_list:
        payload1['numtxt'] = roll
        r1 = s.post("https://oa.cc.iitk.ac.in/Oa/Jsp/OAServices/IITk_SrchRes_new.jsp", headers=headers1, data=payload1)
        soup1 = BeautifulSoup(r1.text, 'html.parser')

        name = ''
        email = ''
        department = ''
        gender = ''

        for para in soup1.select('.TableContent p'):
            body = para.get_text().strip()
            field = body.split(':')
            key = field[0].strip()
            value = field[1].strip()
            if key == 'Name':
                name = value.lower().title()
            elif key == 'E-Mail':
                email = value
            elif key == 'Department':
                department = value
            elif key == 'Gender':
                gender = value[0]

        if name != "" and email != "" and email != "Not Available":
            data = [name, email, department, gender, roll]
            record.append(data)
            user_name = name.split(" ")[0]
            print (name, email, department, gender, roll)

    df = pd.DataFrame(record, columns = ['Name', 'Email', 'Department', 'Gender', 'Roll No']) 
    df.to_csv(filename, encoding='utf-8', index=False)
    return true


## Save Required Roll No in array

In [ ]:
# Save roll numbers in roll_no_list

roll_no_list = []

# for y19 MBA students
for i in range(52):
    roll = 19125001 + i
    roll_no_list.append(roll)

roll_no_list.append(15807678)

## Execute final function

In [6]:
filename = 'student_list.csv'

getRecordsByRollNoList(filename, roll_no_list)